In [1]:
import pandas as pd

In [4]:
data_path = "./Bladder_GSE31189.csv"

In [5]:
data = pd.read_csv(data_path)

In [10]:
name = pd.read_csv("list_output.txt", header=None)

In [8]:
print(data.shape)
print(data.head(2))  # 기본적으로 상위 5개 행 출력
print(data.iloc[:, 0])  # 첫 번째 열
print(data.iloc[:, 2])  # 세 번째 열



(85, 54677)
   samples               type  1007_s_at   1053_at    117_at    121_at   
0      124  tumoral_urothelia   5.448550  2.855473  3.956391  6.990753  \
1      125  tumoral_urothelia   5.948208  2.795529  4.125916  7.085531   

   1255_g_at   1294_at   1316_at   1320_at  ...  AFFX-r2-Ec-bioD-3_at   
0   2.515888  4.532908  5.148095  3.420617  ...             12.355006  \
1   3.161936  4.603227  5.619804  3.194219  ...             12.707785   

   AFFX-r2-Ec-bioD-5_at  AFFX-r2-P1-cre-3_at  AFFX-r2-P1-cre-5_at   
0             12.073772            13.375017            13.129036  \
1             12.626975            14.021214            13.727221   

   AFFX-ThrX-3_at  AFFX-ThrX-5_at  AFFX-ThrX-M_at  AFFX-TrpnX-3_at   
0       14.285014       12.222636       13.764253         2.635243  \
1        8.480108        3.431950        4.462056         2.317526   

   AFFX-TrpnX-5_at  AFFX-TrpnX-M_at  
0         3.087296         2.749863  
1         2.760757         2.868030  

[2 rows x 5

In [11]:

# 라벨링 정보 분리 (samples와 type 열만 따로 저장)
label_data = data[['samples', 'type']]
label_data.to_csv('label_data.csv', index=False)  # 라벨링 정보 저장


# 유전자 발현 데이터만 분리 (samples와 type 열 제외)
gene_expression_data = data.drop(columns=['samples', 'type'])
gene_expression_data.to_csv('gene_expression_data.csv', index=False)  # 유전자 발현 데이터 저장


In [13]:
print(gene_expression_data.head(2))
print(label_data.head(2))

   1007_s_at   1053_at    117_at    121_at  1255_g_at   1294_at   1316_at   
0   5.448550  2.855473  3.956391  6.990753   2.515888  4.532908  5.148095  \
1   5.948208  2.795529  4.125916  7.085531   3.161936  4.603227  5.619804   

    1320_at  1405_i_at   1431_at  ...  AFFX-r2-Ec-bioD-3_at   
0  3.420617   3.378807  3.022539  ...             12.355006  \
1  3.194219   3.287409  2.764392  ...             12.707785   

   AFFX-r2-Ec-bioD-5_at  AFFX-r2-P1-cre-3_at  AFFX-r2-P1-cre-5_at   
0             12.073772            13.375017            13.129036  \
1             12.626975            14.021214            13.727221   

   AFFX-ThrX-3_at  AFFX-ThrX-5_at  AFFX-ThrX-M_at  AFFX-TrpnX-3_at   
0       14.285014       12.222636       13.764253         2.635243  \
1        8.480108        3.431950        4.462056         2.317526   

   AFFX-TrpnX-5_at  AFFX-TrpnX-M_at  
0         3.087296         2.749863  
1         2.760757         2.868030  

[2 rows x 54675 columns]
   samples         

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [20]:
X = gene_expression_data.values
y = label_data['type'].values

In [22]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # 예: tumor -> 0, non_tumor -> 1

In [24]:
print(X)
print(y.shape)

[[5.44855047 2.85547287 3.95639111 ... 2.63524305 3.0872959  2.74986292]
 [5.94820816 2.79552877 4.12591576 ... 2.31752635 2.7607569  2.86802983]
 [5.49623909 2.88156934 4.07656101 ... 2.32313295 2.69599462 2.6706242 ]
 ...
 [6.18813431 2.9007399  4.73405634 ... 2.1427781  2.38023166 2.29607142]
 [6.35865158 3.43926188 3.63315948 ... 2.16832783 2.30002383 2.28086162]
 [6.2930911  2.87422621 4.02052646 ... 2.1292677  2.28813348 2.32520466]]
(85,)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [46]:
print(X_train.shape)
print(y_train.shape)

(59, 54675)
(59,)


In [52]:
rf = RandomForestClassifier(n_estimators=2, random_state=42)
rf.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = rf.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 65.38%


In [50]:

# 평가
print("훈련 세트 정확도: {:.3f}".format(rf.score(X_train, y_train)) )
print("테스트 세트 정확도: {:.3f}".format(rf.score(X_test, y_test)) )


훈련 세트 정확도: 1.000
테스트 세트 정확도: 0.538


In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define parameter grid for RandomForestClassifier
params = {
    'n_estimators': [50, 100, 200],          # Number of trees in the forest
    'max_depth': [5, 10, 15, None],          # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],         # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],           # Minimum samples required at each leaf node
    'bootstrap': [True, False]               # Whether bootstrap samples are used when building trees
}

# Create RandomForestClassifier model
rf_clf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the model with training data
grid_cv.fit(X_train, y_train)

# Get the best estimator and predict on the validation set
best_rf_clf = grid_cv.best_estimator_
y_pred = best_rf_clf.predict(X_test)

# Print the best parameters and the best score
print("최적 하이퍼 파라미터:\n", grid_cv.best_params_)
print("최고 예측 정확도: {0:.4f}".format(grid_cv.best_score_))

# Evaluate the model on validation data
accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits


[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=5, min_samples_leaf=4, min_samples_split=1

In [27]:
print(label_encoder.classes_)


['normal_urothelia' 'tumoral_urothelia']
